In [3]:
import pandas as pd
import numpy as np
from math import log
from random import randrange
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

In [4]:
# read all data from train
train_data = []
train_label = []
for i in range(1, 6):
    tmp = pd.read_csv('./data/train%d.csv' % i, header=None)
    label = pd.read_csv('./data/label%d.csv' % i, header=None)
    train_data.append(tmp)
    train_label.append(label)
x = train_data[0]
y = train_label[0]
for i in range(4):
    x = np.vstack((x, train_data[i]))
    y = np.vstack((y, train_label[i]))
dataset = np.hstack((x, y))
kf = KFold(n_splits=5)
k_fold_data = []
for tra_idx, val_idx in kf.split(dataset):
    tra = dataset[tra_idx]
    val = dataset[val_idx]
    k_fold_data.append((tra, val))
    print(len(tra) + len(val))

10000003
10000003
10000003
10000003
10000003


In [28]:
def test_split(index, value, dataset):
    """
    description: split the current dataset by the `index`th feature and value
    """
    left, right = [], []
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def gini_index(groups, class_values):
    """
    description: evaluate the gini value of the current groups, 
    containing the left group and the right group, splited by the nth feature.
    `class_values` are all the possible values in the label column.
    """
    gini = 0.0
    for class_value in class_values:
        for group in groups:
            if len(group) == 0:
                continue
            proportion = [row[-1] for row in group].count(class_value) / len(group)
            gini += (proportion * (1.0 - proportion))
    return gini

def to_terminal(group):
    """
    description: terminate the split and set the most frequent value as the output
    """
    output = [row[-1] for row in group]
    return max(set(output), key=output.count)

def get_split(dataset, n_features):
    """
    description: choose a feature, which gains the highest gini value,
    to split the dataset.
    """
    class_values = [row[-1] for row in dataset]
    s_index, s_value, s_score, s_groups = 999999, 999999, 999999, None
    cur_features = [] # choose partition feature randomly
    while len(cur_features) < log(2, n_features):
        index = randrange(len(dataset[0] - 1))
        if index not in cur_features:
            cur_features.append(index)
    for index in cur_features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < s_score:
                s_index, s_value, s_score, s_groups = index, row[index], gini, groups
    return { 'index': s_index, 'value': s_value, 'group': s_groups }

def split(node, max_depth, min_size, n_features, depth):
    """
    description: split the node recursively.
    """
    left, right = node['group']
    del(node['group'])
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth + 1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth + 1)

def build_tree(train, max_depth, min_size, n_features):
    """
    description: build a tree
    """
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

def predict(node, row):
    """
    description: make prediction with the tree, navigating it and get the output
    """
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

def sub_sample(dataset, ratio):
    """
    description: select the data ramdomly.
    """
    sample = []
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

In [29]:
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = []
    for i in range(n_trees):
        sample = sub_sample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        print('finish')
        trees.append(tree)
    all_predict = []
    all_y = []
    for row in test:
        predictions = [predict(tree, row) for tree in trees]
        cur_predict = max(set(predictions), key=predictions.count)
        all_y.append(row[-1])
        all_predict.append(cur_predict)
    return all_predict, r2_score(all_y, all_predict)

In [30]:
for i in range(5):
    (train, test) = k_fold_data[i]
    # train = train[:10000]
    # test = test[:100]
    _, score = random_forest(train, test, 5, 1, 0.8, 1, train[0].shape[0] - 1)
    print(score)

finish
-0.3049475129155783
finish
-0.2787080208555386
finish
-0.0502273329390579
finish
-0.16307483454946547
finish
-0.060151858728111574


In [30]:
from random import randint
from multiprocessing import Lock, Queue, Pool
lock = Lock()
result = Queue()
def append_tree(sample, max_depth, min_size, n_features):
    tree = build_tree(sample, max_depth, min_size, n_features)
    result.put(tree)   
p = Pool()
p.close()
p.join()


def multi_process_random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = []
    for i in range(n_trees):
        sample = sub_sample(train, sample_size)
        p.apply_async(build_tree, args=(sample, max_depth, min_size, n_features,))
    p.close()
    p.join()
    trees = []
    while not result.empty():
        trees.append(result.get())
    all_predict = []
    all_y = []
    for row in test:
        predictions = [predict(tree, row) for tree in trees]
        cur_predict = max(set(predictions), key=predictions.count)
        all_y.append(row[-1])
        all_predict.append(cur_predict)
    return all_predict, r2_score(all_y, all_predict)